In [1]:
!pip install -U skorch

     |████████████████████████████████| 112kB 4.8MB/s 


In [2]:
from skorch import NeuralNetRegressor, NeuralNetClassifier, NeuralNet
import skorch
from skorch.dataset import CVSplit
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split as ttsplit
from sklearn import tree
from sklearn import linear_model
from sklearn import svm
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression as LR
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split as ttsplit
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import cat
import torch.optim as optim
from torch.utils import data as data_torch
from torch.autograd import Variable
import matplotlib.pyplot as plt
import os

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
ours = 'drive/My Drive/CS 598PS Final Project/data/our_data.csv' 
best = 'drive/My Drive/CS 598PS Final Project/data/best_score_data.csv'

def get_data(f):
  data_wTarget = pd.read_csv(f, header=0, index_col=0)
  data = data_wTarget[[col for col in data_wTarget.columns if col != 'Yards']]
  target = data_wTarget['Yards']
  data_np = data.to_numpy(dtype='float')
  target_np = target.to_numpy(dtype='int')

  # SSscaler = StandardScaler()
  # data_np  = SSscaler.fit_transform(data_np)
  return data, target

our_data, our_target = get_data(ours)
best_data, best_target = get_data(best)


In [5]:
list(best_data.columns)


['GameId',
 'PlayId',
 'back_from_scrimmage',
 'back_oriented_down_field',
 'back_moving_down_field',
 'min_dist',
 'max_dist',
 'mean_dist',
 'std_dist',
 'def_min_dist',
 'def_max_dist',
 'def_mean_dist',
 'def_std_dist',
 'RusherDir',
 'RusherS',
 'RusherA',
 'RusherX',
 'RusherY',
 'v_horizontal',
 'v_vertical',
 'PlayerHeight_dense',
 'PlayerAge',
 'WindSpeed_dense',
 'Dir_sin',
 'Dir_cos',
 'diffScoreBeforePlay',
 'X',
 'Y',
 'S',
 'A',
 'Dis',
 'Orientation',
 'Dir',
 'YardLine',
 'Quarter',
 'Down',
 'Distance',
 'DefendersInTheBox']

In [0]:
# pd.DataFrame(our_data).head()

In [7]:
best_data = best_data.drop(columns='GameId', errors='ignore')
best_data = best_data.drop(columns='PlayId', errors='ignore')
pd.DataFrame(best_data).head()

,back_from_scrimmage,back_oriented_down_field,back_moving_down_field,min_dist,max_dist,mean_dist,std_dist,def_min_dist,def_max_dist,def_mean_dist,def_std_dist,RusherDir,RusherS,RusherA,RusherX,RusherY,v_horizontal,v_vertical,PlayerHeight_dense,PlayerAge,WindSpeed_dense,Dir_sin,Dir_cos,diffScoreBeforePlay,X,Y,S,A,Dis,Orientation,Dir,YardLine,Quarter,Down,Distance,DefendersInTheBox
0,-33.75,0,1,1.449724,22.415872,8.046559,4.873845,4.593310,22.415872,9.752491,5.327299,245.74,3.63,3.35,78.75,30.53,3.309436,1.491487,70,25.596251,8.0,-0.911690,-0.410878,0,78.75,30.53,3.63,3.35,0.38,161.98,245.74,45.0,1,3,2,6.0
1,-18.07,1,1,0.792023,23.025872,8.614623,5.598683,4.287773,23.025872,10.297028,5.833217,312.20,3.06,2.41,71.07,27.16,2.266862,2.055465,70,25.596252,8.0,-0.740805,0.671721,0,71.07,27.16,3.06,2.41,0.34,210.70,312.20,53.0,1,1,10,6.0
2,26.34,0,1,1.646390,20.726285,8.482583,4.642121,4.221670,20.726285,9.903689,5.073290,221.96,5.77,2.42,48.66,19.11,3.857889,4.290640,70,25.596253,8.0,-0.668612,-0.743612,0,48.66,19.11,5.77,2.42,0.60,140.82,221.96,75.0,1,1,10,7.0
3,92.47,1,1,0.918096,9.791231,5.549379,1.983128,4.528002,9.791231,6.309354,1.834174,275.44,4.45,3.20,15.53,25.36,4.429957,0.421875,71,26.852933,8.0,-0.995496,0.094803,0,15.53,25.36,4.45,3.20,0.46,186.22,275.44,108.0,1,2,2,9.0
4,5.01,0,0,0.502892,21.214806,9.168819,5.611232,4.288088,21.214806,11.056456,5.900009,157.92,3.90,2.53,29.99,27.12,1.466013,3.613974,71,22.091819,8.0,0.375901,-0.926660,7,29.99,27.12,3.90,2.53,0.44,34.27,157.92,35.0,1,1,10,7.0


In [8]:
best_data

,back_from_scrimmage,back_oriented_down_field,back_moving_down_field,min_dist,max_dist,mean_dist,std_dist,def_min_dist,def_max_dist,def_mean_dist,def_std_dist,RusherDir,RusherS,RusherA,RusherX,RusherY,v_horizontal,v_vertical,PlayerHeight_dense,PlayerAge,WindSpeed_dense,Dir_sin,Dir_cos,diffScoreBeforePlay,X,Y,S,A,Dis,Orientation,Dir,YardLine,Quarter,Down,Distance,DefendersInTheBox
0,-33.75,0,1,1.449724,22.415872,8.046559,4.873845,4.593310,22.415872,9.752491,5.327299,245.74,3.63,3.35,78.75,30.53,3.309436,1.491487,70,25.596251,8.0,-0.911690,-0.410878,0,78.75,30.53,3.63,3.35,0.38,161.98,245.74,45.0,1,3,2,6.0
1,-18.07,1,1,0.792023,23.025872,8.614623,5.598683,4.287773,23.025872,10.297028,5.833217,312.20,3.06,2.41,71.07,27.16,2.266862,2.055465,70,25.596252,8.0,-0.740805,0.671721,0,71.07,27.16,3.06,2.41,0.34,210.70,312.20,53.0,1,1,10,6.0
2,26.34,0,1,1.646390,20.726285,8.482583,4.642121,4.221670,20.726285,9.903689,5.073290,221.96,5.77,2.42,48.66,19.11,3.857889,4.290640,70,25.596253,8.0,-0.668612,-0.743612,0,48.66,19.11,5.77,2.42,0.60,140.82,221.96,75.0,1,1,10,7.0
3,92.47,1,1,0.918096,9.791231,5.549379,1.983128,4.528002,9.791231,6.309354,1.834174,275.44,4.45,3.20,15.53,25.36,4.429957,0.421875,71,26.852933,8.0,-0.995496,0.094803,0,15.53,25.36,4.45,3.20,0.46,186.22,275.44,108.0,1,2,2,9.0
4,5.01,0,0,0.502892,21.214806,9.168819,5.611232,4.288088,21.214806,11.056456,5.900009,157.92,3.90,2.53,29.99,27.12,1.466013,3.613974,71,22.091819,8.0,0.375901,-0.926660,7,29.99,27.12,3.90,2.53,0.44,34.27,157.92,35.0,1,1,10,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23166,25.43,1,1,0.575847,19.341419,6.736412,4.514974,2.942329,19.341419,8.243270,5.075063,290.23,3.99,3.38,49.57,32.27,3.743865,1.379700,73,27.041779,5.0,-0.938312,0.345790,3,49.57,32.27,3.99,3.38,0.39,287.44,290.23,75.0,4,1,10,8.0
23167,4.00,0,0,1.151217,25.500196,9.827962,7.474030,2.808932,25.500196,11.779213,7.739310,150.98,4.18,2.95,31.00,30.99,2.027780,3.655203,69,25.861781,5.0,0.485115,-0.874450,0,31.00,30.99,4.18,2.95,0.40,122.97,150.98,35.0,4,1,10,6.0
23168,4.07,0,0,0.776209,19.454503,8.272970,5.548527,3.595998,19.454503,8.825731,5.317643,41.25,4.67,3.68,39.93,29.20,3.079145,3.511092,69,25.861783,5.0,0.659346,0.751840,0,39.93,29.20,4.67,3.68,0.45,63.11,41.25,44.0,4,3,1,7.0
23169,3.81,0,0,0.584637,19.113474,8.202171,5.833721,4.090880,19.113474,9.062557,5.568974,41.24,4.23,2.43,81.19,23.75,2.788478,3.180769,71,24.290258,5.0,0.659215,0.751955,0,81.19,23.75,4.23,2.43,0.41,66.34,41.24,85.0,4,1,10,7.0


In [9]:
all(our_target == best_target)

True

In [10]:
[x for x in our_data if x in best_data]

['YardLine', 'Distance', 'DefendersInTheBox']

In [11]:
list(our_data.columns)

['YardLine',
 'Distance',
 'DefendersInTheBox',
 'Week',
 'GameClock_in_seconds',
 'in_own_half',
 'home_team_has_poss',
 'score_delta',
 'Formation_ACE',
 'Formation_EMPTY',
 'Formation_I_FORM',
 'Formation_JUMBO',
 'Formation_PISTOL',
 'Formation_SHOTGUN',
 'Formation_SINGLEBACK',
 'Formation_WILDCAT',
 'offensive_personnel_LB',
 'offensive_personnel_WR',
 'offensive_personnel_RB',
 'offensive_personnel_TE',
 'offensive_personnel_OL',
 'offensive_personnel_QB',
 'offensive_personnel_DL',
 'offensive_personnel_DB',
 'defensive_personnel_DL',
 'defensive_personnel_LB',
 'defensive_personnel_OL',
 'defensive_personnel_DB',
 'play_dir_is_left',
 'TimeDelta',
 'X1',
 'Y1',
 'S1',
 'A1',
 'Dis1',
 'Orientation1',
 'Dir1',
 'PlayerWeight1',
 'is_rusher1',
 'jersey_group1',
 'Player_Height_Inches1',
 'X2',
 'Y2',
 'S2',
 'A2',
 'Dis2',
 'Orientation2',
 'Dir2',
 'PlayerWeight2',
 'is_rusher2',
 'jersey_group2',
 'Player_Height_Inches2',
 'X3',
 'Y3',
 'S3',
 'A3',
 'Dis3',
 'Orientation3',
 

In [0]:
to_keep = [
  'GameClock_in_seconds',
  'in_own_half',
  'home_team_has_poss',
  'Formation_ACE',
  'Formation_EMPTY',
  'Formation_I_FORM',
  'Formation_JUMBO',
  'Formation_PISTOL',
  'Formation_SHOTGUN',
  'Formation_SINGLEBACK',
  'Formation_WILDCAT',
  'play_dir_is_left',
  'TimeDelta',
  'Quarter_1',
  'Quarter_2',
  'Quarter_3',
  'Quarter_4',
  'Quarter_5',
  'Down_1',
  'Down_2',
  'Down_3',
  'Down_4',
 ]

In [13]:
 our_data = our_data[to_keep]
 list(our_data.columns)

['GameClock_in_seconds',
 'in_own_half',
 'home_team_has_poss',
 'Formation_ACE',
 'Formation_EMPTY',
 'Formation_I_FORM',
 'Formation_JUMBO',
 'Formation_PISTOL',
 'Formation_SHOTGUN',
 'Formation_SINGLEBACK',
 'Formation_WILDCAT',
 'play_dir_is_left',
 'TimeDelta',
 'Quarter_1',
 'Quarter_2',
 'Quarter_3',
 'Quarter_4',
 'Quarter_5',
 'Down_1',
 'Down_2',
 'Down_3',
 'Down_4']

In [14]:
list(best_data.columns)

['back_from_scrimmage',
 'back_oriented_down_field',
 'back_moving_down_field',
 'min_dist',
 'max_dist',
 'mean_dist',
 'std_dist',
 'def_min_dist',
 'def_max_dist',
 'def_mean_dist',
 'def_std_dist',
 'RusherDir',
 'RusherS',
 'RusherA',
 'RusherX',
 'RusherY',
 'v_horizontal',
 'v_vertical',
 'PlayerHeight_dense',
 'PlayerAge',
 'WindSpeed_dense',
 'Dir_sin',
 'Dir_cos',
 'diffScoreBeforePlay',
 'X',
 'Y',
 'S',
 'A',
 'Dis',
 'Orientation',
 'Dir',
 'YardLine',
 'Quarter',
 'Down',
 'Distance',
 'DefendersInTheBox']

In [0]:
to_remove = [
  # 'PlayerHeight_dense', 
  # 'PlayerAge', 
  # 'X',
  # 'Y',
  # 'S',
  # 'A',
]

In [0]:
best_data = best_data[[x for x in best_data.columns if x not in to_remove]]

There are not too many duplicated features. Let's try simply combine them.

In [17]:
combined = pd.concat([our_data, best_data, our_target], axis=1)
combined

,GameClock_in_seconds,in_own_half,home_team_has_poss,Formation_ACE,Formation_EMPTY,Formation_I_FORM,Formation_JUMBO,Formation_PISTOL,Formation_SHOTGUN,Formation_SINGLEBACK,Formation_WILDCAT,play_dir_is_left,TimeDelta,Quarter_1,Quarter_2,Quarter_3,Quarter_4,Quarter_5,Down_1,Down_2,Down_3,Down_4,back_from_scrimmage,back_oriented_down_field,back_moving_down_field,min_dist,max_dist,mean_dist,std_dist,def_min_dist,def_max_dist,def_mean_dist,def_std_dist,RusherDir,RusherS,RusherA,RusherX,RusherY,v_horizontal,v_vertical,PlayerHeight_dense,PlayerAge,WindSpeed_dense,Dir_sin,Dir_cos,diffScoreBeforePlay,X,Y,S,A,Dis,Orientation,Dir,YardLine,Quarter,Down,Distance,DefendersInTheBox,Yards
0,854,True,True,0,0,0,0,0,1,0,0,True,1.0,1,0,0,0,0,0,0,1,0,-33.75,0,1,1.449724,22.415872,8.046559,4.873845,4.593310,22.415872,9.752491,5.327299,245.74,3.63,3.35,78.75,30.53,3.309436,1.491487,70,25.596251,8.0,-0.911690,-0.410878,0,78.75,30.53,3.63,3.35,0.38,161.98,245.74,45.0,1,3,2,6.0,8
1,832,True,True,0,0,0,0,0,1,0,0,True,1.0,1,0,0,0,0,1,0,0,0,-18.07,1,1,0.792023,23.025872,8.614623,5.598683,4.287773,23.025872,10.297028,5.833217,312.20,3.06,2.41,71.07,27.16,2.266862,2.055465,70,25.596252,8.0,-0.740805,0.671721,0,71.07,27.16,3.06,2.41,0.34,210.70,312.20,53.0,1,1,10,6.0,3
2,782,False,True,0,0,0,0,0,0,1,0,True,2.0,1,0,0,0,0,1,0,0,0,26.34,0,1,1.646390,20.726285,8.482583,4.642121,4.221670,20.726285,9.903689,5.073290,221.96,5.77,2.42,48.66,19.11,3.857889,4.290640,70,25.596253,8.0,-0.668612,-0.743612,0,48.66,19.11,5.77,2.42,0.60,140.82,221.96,75.0,1,1,10,7.0,5
3,732,False,True,0,0,0,1,0,0,0,0,True,2.0,1,0,0,0,0,0,1,0,0,92.47,1,1,0.918096,9.791231,5.549379,1.983128,4.528002,9.791231,6.309354,1.834174,275.44,4.45,3.20,15.53,25.36,4.429957,0.421875,71,26.852933,8.0,-0.995496,0.094803,0,15.53,25.36,4.45,3.20,0.46,186.22,275.44,108.0,1,2,2,9.0,2
4,728,True,False,0,0,0,0,0,1,0,0,False,1.0,1,0,0,0,0,1,0,0,0,5.01,0,0,0.502892,21.214806,9.168819,5.611232,4.288088,21.214806,11.056456,5.900009,157.92,3.90,2.53,29.99,27.12,1.466013,3.613974,71,22.091819,8.0,0.375901,-0.926660,7,29.99,27.12,3.90,2.53,0.44,34.27,157.92,35.0,1,1,10,7.0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23166,183,False,False,0,0,1,0,0,0,0,0,True,1.0,0,0,0,1,0,1,0,0,0,25.43,1,1,0.575847,19.341419,6.736412,4.514974,2.942329,19.341419,8.243270,5.075063,290.23,3.99,3.38,49.57,32.27,3.743865,1.379700,73,27.041779,5.0,-0.938312,0.345790,3,49.57,32.27,3.99,3.38,0.39,287.44,290.23,75.0,4,1,10,8.0,1
23167,109,True,True,0,0,0,0,0,1,0,0,False,1.0,0,0,0,1,0,1,0,0,0,4.00,0,0,1.151217,25.500196,9.827962,7.474030,2.808932,25.500196,11.779213,7.739310,150.98,4.18,2.95,31.00,30.99,2.027780,3.655203,69,25.861781,5.0,0.485115,-0.874450,0,31.00,30.99,4.18,2.95,0.40,122.97,150.98,35.0,4,1,10,6.0,4
23168,84,True,True,0,0,0,0,0,1,0,0,False,2.0,0,0,0,1,0,0,0,1,0,4.07,0,0,0.776209,19.454503,8.272970,5.548527,3.595998,19.454503,8.825731,5.317643,41.25,4.67,3.68,39.93,29.20,3.079145,3.511092,69,25.861783,5.0,0.659346,0.751840,0,39.93,29.20,4.67,3.68,0.45,63.11,41.25,44.0,4,3,1,7.0,4
23169,56,False,True,0,0,0,0,0,1,0,0,False,1.0,0,0,0,1,0,1,0,0,0,3.81,0,0,0.584637,19.113474,8.202171,5.833721,4.090880,19.113474,9.062557,5.568974,41.24,4.23,2.43,81.19,23.75,2.788478,3.180769,71,24.290258,5.0,0.659215,0.751955,0,81.19,23.75,4.23,2.43,0.41,66.34,41.24,85.0,4,1,10,7.0,2


In [0]:
combined.to_csv('drive/My Drive/CS 598PS Final Project/data/combined_data_smarter_2.csv')

In [19]:
print(set(combined.columns) ^ set(best_data.columns))

{'Down_3', 'Down_4', 'Formation_ACE', 'Quarter_3', 'home_team_has_poss', 'play_dir_is_left', 'Quarter_5', 'Down_1', 'Formation_SINGLEBACK', 'in_own_half', 'GameClock_in_seconds', 'Formation_EMPTY', 'Formation_JUMBO', 'Formation_WILDCAT', 'Quarter_2', 'Formation_SHOTGUN', 'Quarter_4', 'Down_2', 'Yards', 'Formation_I_FORM', 'Quarter_1', 'TimeDelta', 'Formation_PISTOL'}


In [20]:
list(combined.columns)

['GameClock_in_seconds',
 'in_own_half',
 'home_team_has_poss',
 'Formation_ACE',
 'Formation_EMPTY',
 'Formation_I_FORM',
 'Formation_JUMBO',
 'Formation_PISTOL',
 'Formation_SHOTGUN',
 'Formation_SINGLEBACK',
 'Formation_WILDCAT',
 'play_dir_is_left',
 'TimeDelta',
 'Quarter_1',
 'Quarter_2',
 'Quarter_3',
 'Quarter_4',
 'Quarter_5',
 'Down_1',
 'Down_2',
 'Down_3',
 'Down_4',
 'back_from_scrimmage',
 'back_oriented_down_field',
 'back_moving_down_field',
 'min_dist',
 'max_dist',
 'mean_dist',
 'std_dist',
 'def_min_dist',
 'def_max_dist',
 'def_mean_dist',
 'def_std_dist',
 'RusherDir',
 'RusherS',
 'RusherA',
 'RusherX',
 'RusherY',
 'v_horizontal',
 'v_vertical',
 'PlayerHeight_dense',
 'PlayerAge',
 'WindSpeed_dense',
 'Dir_sin',
 'Dir_cos',
 'diffScoreBeforePlay',
 'X',
 'Y',
 'S',
 'A',
 'Dis',
 'Orientation',
 'Dir',
 'YardLine',
 'Quarter',
 'Down',
 'Distance',
 'DefendersInTheBox',
 'Yards']